In [11]:
from random import randint
class SearchEntry():
    def __init__(self, x, y, z, g_cost, c_cost, lc_cost ,f_cost=0, pre_entry=None):
        self.x = x
        self.y = y
        self.z = z
        # cost move form start entry to this entry
        self.g_cost = g_cost
        self.c_cost = c_cost
        self.lc_cost = lc_cost
        self.f_cost = f_cost
        self.pre_entry = pre_entry

    def getPos(self):
        return (self.x, self.y, self.z)

class Map():
    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth
        self.map = [[[" " for x in range(self.width)] for y in range(self.height)] for z in range(self.depth)]

    def setBlock(self,block_list): #block list 储存已经存在的单元位置信息
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            z=block_list[i][2]
            self.map[z][y][x] = z; 
    def setBlock_N(self,block_list): #block list 储存已经存在的单元位置信息
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            z=block_list[i][2]
            self.map[z][y][x] = "N"; 
    def setBlock_test(self,block_list,index): #block list 储存已经存在的单元位置信息
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            z=block_list[i][2]
            self.map[index][y][x] = z; 
                
    
    def clrBlock(self,block_list): #清除block list
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            z=block_list[i][2]
            self.map[z][y][x] = " ";
            
    def get_blocked_point(self,block_list):#用来检查block_list里的点是否原来就是封死的，返回一个原来是封死的list，防止clrblock时把原来的也清掉了
        block_list_len=len(block_list)
        prev_block=[]
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            z=block_list[i][2]
            if(self.map[z][y][x]==1):
                prev_block.append([x,y,z])
        return prev_block

    
    def showMap(self,z):
        print("+" * (3 * self.width + 2))
        len_row=len(self.map[z])
        #print(len_row)
        for i in range(len_row):
            s = '+'
            len_col=len(self.map[z][len_row-i-1])
            for j in range(len_col):
                if(self.map[z][len_row-i-1][j]==0):
                    s +=' '+ "0" + ' '
                else:
                    s +=' '+ str(self.map[z][len_row-i-1][j]) + ' '                   
            s +='+'
            print(s)

        print("+" * (3 * self.width + 2))
        
    def createBlock(self, block_num,z):
        for i in range(block_num):
            x, y = (randint(0, self.width-1), randint(0, self.height-1))
            self.map[z][y][x] = "N"

In [32]:
def AStarSearch(map, source, dest):
    def getNewPosition(map, locatioin, offset):
        x,y,z = (location.x + offset[0], location.y + offset[1], location.z + offset[2])
        pos=(x,y,z)
        if x < 0 or x >= map.width or y < 0 or y >= map.height or z<0 or z>=map.depth or map.map[z][y][x] == z or  map.map[z][y][x] == "N":
            pos=None
            
        else:
            if(location.c_cost != 0):
                if(offset[2]!=0):
                    pos=None
                
            if(location.pre_entry != None and location.pre_entry.pre_entry != None):
                if(location.pre_entry.pre_entry.x==x and location.pre_entry.pre_entry.y==y):
                    pos=None
                if(location.pre_entry.z != location.pre_entry.pre_entry.z):
                    if(abs(location.pre_entry.x-x)==1 and abs(location.pre_entry.y-1)==1):
                        pos=None
                    elif(offset[2]!=0):
                        pos=None
        return pos

    def getPositions(map, location):#这里设置寻路类型，使用曼哈顿或折线距离
        # use four ways or eight ways to move 
        if(location.pre_entry != None and location.pre_entry.pre_entry != None):
            if(location.pre_entry.z != location.pre_entry.pre_entry.z):
                if(abs(location.pre_entry.x-location.x)==1): 
                    offsets = [(-1,0,0),  (1,0,0)]
                elif(abs(location.pre_entry.y-location.y)==1): 
                    offsets = [(0,1,0),  (0,-1,0)]
            else:
                offsets = [(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]
        elif(location.pre_entry != None):
            offsets = [(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]

                
        else:
            offsets = [(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]
        poslist = []
        for offset in offsets:
            pos = getNewPosition(map, location, offset)
            if pos is not None:
                poslist.append(pos)
        return poslist

    # imporve the heuristic distance more precisely in future
    def calHeuristic(pos, dest):
        return abs(dest.x - pos[0]) + abs(dest.y - pos[1])
        
    def getMoveCost(location, pos):
        if location.x != pos[0] and location.y != pos[1]:
            return 1.4
        else:
            return 1

    def getCorner(next_pos,location):
        corner_cost=10
        lc_corner_cost=1000
        if(location.pre_entry==None):
            return 0
        else:
            if(location.z!=location.pre_entry.z):
                if(location.pre_entry.pre_entry==None):
                    return corner_cost
                else:
                    if(location.pre_entry.pre_entry.z != location.pre_entry.z):
                        delta_x=abs(location.pre_entry.x-next_pos[0])
                        delta_y=abs(location.pre_entry.y-next_pos[1])
                        if(delta_x==1 and delta_y==1):
                            return lc_corner_cost
                        else:
                            return 0  
                    else:
                        delta_x=abs(location.pre_entry.pre_entry.x-next_pos[0])
                        delta_y=abs(location.pre_entry.pre_entry.y-next_pos[1])
                        if(delta_x==1 and delta_y==1):
                            return lc_corner_cost
                        else:
                            return 0
            #elif(location.pre_entry.pre_entry != None):
            #    if(location.pre_entry.pre_entry.z !=location.pre_entry.z):
            #        return lc_corner_cost
            #    else:
            #        return 0
            else:
                delta_x=abs(location.pre_entry.x-next_pos[0])
                delta_y=abs(location.pre_entry.y-next_pos[1])
                if(delta_x==1 and delta_y==1):
                    return corner_cost
                else:
                    return 0

                
    def getLayer(location,pos,depth):
        layer=location.z
        change_cost=0
        corner_or_layerc = 1
        if(location.pre_entry != None and location.pre_entry.z != location.z):
            if(location.z!=pos[2]):
                change_cost=1000
        elif(location.z!=pos[2]):
            change_cost=1
        layer_cost=0
        if(layer==0):
            layer_cost = 8
        elif(layer==1):
            layer_cost = 7
        elif(layer==2):
            layer_cost = 5
        elif(layer==3):
            layer_cost = 1
        elif(layer==4):
            layer_cost = 0.5
        elif(layer==5):
            layer_cost = 1
        return [layer_cost+change_cost,change_cost]
    def isInList(list, pos):
        if pos in list:
            return list[pos]
        return None
    
    # add available adjacent positions
    def addAdjacentPositions(map, location, dest, openlist, closedlist):
        poslist = getPositions(map, location)
        for pos in poslist:
            # if position is already in closedlist, do nothing
            if isInList(closedlist, pos) is None:
                findEntry = isInList(openlist, pos)
                h_cost = calHeuristic(pos, dest)
                [l_cost,change_cost] = getLayer(location,pos,map.depth)
                c_cost = getCorner(pos,location)
                g_cost = location.g_cost + getMoveCost(location, pos)+c_cost+l_cost
                if(location.pre_entry != None and location.pre_entry.lc_cost != 0):
                    print("location:")
                    print(location.x,location.y,location.z)
                    print("poslist")
                    print(poslist)
                if findEntry is None :
                    # if position is not in openlist, add it to openlist
                    openlist[pos] = SearchEntry(pos[0], pos[1],pos[2], g_cost, c_cost,change_cost,g_cost+h_cost, location)
                elif findEntry.g_cost > g_cost:
                    # if position is in openlist and cost is larger than current one,
                    # then update cost and previous position
                    findEntry.g_cost = g_cost
                    findEntry.c_cost = c_cost
                    findEntry.lc_cost = change_cost
                    findEntry.f_cost = g_cost + h_cost
                    findEntry.pre_entry = location

    # find a least cost position in openlist, return None if openlist is empty
    def getFastPosition(openlist):
        fast = None
        for entry in openlist.values():
            #print(fast,entry)
            if fast is None:
                fast = entry
            #else:
            #    print(fast.f_cost)
            elif fast.f_cost > entry.f_cost:
                fast = entry
        return fast

    openlist = {}
    closedlist = {}
    location = SearchEntry(source[0], source[1],source[2], 0.0,0.0,0.0)
    dest = SearchEntry(dest[0], dest[1],dest[2], 0.0, 0.0, 0.0)
    openlist[source] = location

    while True:
        location = getFastPosition(openlist)
        #print(location)
        if location is None:
            # not found valid path
            print("can't find valid path")
            break;
        
        if location.x == dest.x and location.y == dest.y and location.z == dest.z:
            break

        closedlist[location.getPos()] = location
        openlist.pop(location.getPos())
        addAdjacentPositions(map, location, dest, openlist, closedlist)

    #mark the found path at the map
    path_temp=[]
    while location is not None:
        #print(location.x,location.y)
        path_temp.append([location.x,location.y,location.z])
        location = location.pre_entry
    path=path_temp[::-1]
    return path

In [34]:
#测试模块
import time
map_width=4
map_height=3
map_depth=2
map2=Map(map_width,map_height,map_depth)
map3=Map(map_width,map_height,1)



map2.setBlock_N([[0,0,1],[1,0,1],[2,0,1],[3,0,1],[0,2,1],[1,2,1],[2,2,1],[3,2,1],[2,1,1],[3,1,1]])
#map2.setBlock_N([[3,1,0]])

#map2.setBlock([[37,38],[36,38],[35,38],[38,38],[39,38],[33,39],[34,37],[33,37]])
#map2.setBlock([[37,1],[36,1],[35,1],[38,1],[39,1],[39,3],[37,3],[38,3],[36,3],[35,3]])

source1=(0,1,1)
dest1=(3,0,0)

#map2.clrBlock([source1,dest1])



start=time.time()
path1=AStarSearch(map2,source1,dest1)


map2.setBlock(path1)

end=time.time()
print(end-start)

for i in range(map_depth):
    map2.showMap(i)



location:
2 1 0
poslist
[(3, 1, 0)]
0.0009233951568603516
++++++++++++++
+            +
+    0  0  0 +
+          0 +
++++++++++++++
++++++++++++++
+ N  N  N  N +
+ 1  1  N  N +
+ N  N  N  N +
++++++++++++++
